In [103]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

### Segédfüggvények

In [104]:
def limitalt_dummy_valtozo(indf, oszlop, limit, drop_last=True):
    uj_valtozok = []
    df = indf.copy()
    egyedi_elemszam=df[oszlop].nunique()
    if egyedi_elemszam <= limit and drop_last is True:
        egyedi_elemszam = egyedi_elemszam - 1
    tmp = df[oszlop].value_counts()[:limit]
    for value in list(tmp.index):
        def f(x):
            if x == value:
                return 1
            else:
                return 0
        df[oszlop+'='+str(value)] = df[oszlop].apply(f)
        uj_valtozok.append(oszlop + '=' + str(value))
    return df, uj_valtozok

In [105]:
def myxval(model,train_df,bemeno_valtozok,fold_num=3):
    train_df['xval']=train_df['ID']%fold_num
    auc_list=[]
    for i in range(fold_num):
        mini_train=train_df[train_df['xval']!=i].copy()
        mini_test=train_df[train_df['xval']==i].copy()
        model.fit(mini_train[bemeno_valtozok],mini_train['TARGET'])
        mini_test['p1']=model.predict_proba(mini_test[bemeno_valtozok])[:,1]
        auc=roc_auc_score(mini_test['TARGET'],mini_test['p1'])
        auc_list.append(auc)
    return np.mean(auc_list)

In [107]:
from sklearn.metrics import roc_auc_score

def myxval_multi(models,train_df,bemeno_valtozok,fold_num=3):
    train_df['xval']=train_df['ID']%fold_num
    auc_list=[]
    for i in range(fold_num):
        mini_train=train_df[train_df['xval']!=i].copy()
        mini_test=train_df[train_df['xval']==i].copy()
        predict_list = None
        for model in models:
            model.fit(mini_train[bemeno_valtozok],mini_train['TARGET'])
            predictions=model.predict_proba(mini_test[bemeno_valtozok])[:,1]
            predictions = predictions.reshape((1, len(predictions)))
            if predict_list is None:
                predict_list = predictions
            else:
                predict_list = np.concatenate([predict_list, predictions], axis=0)
        mini_test['p1'] = np.mean(predict_list, axis=0)
        auc=roc_auc_score(mini_test['TARGET'],mini_test['p1'])
        auc_list.append(auc)
    return np.mean(auc_list)

### --------------------------------------------------------------------- ###

In [108]:
train_data = pd.read_csv('./data/public_train.csv')
test_data = pd.read_csv('./data/public_test.csv')

C:\Programs\miniconda\envs\dap2020_jupyter\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [109]:
# Train or test?
train_data['train_or_test'] = 'train'
test_data['train_or_test'] = 'test'

In [110]:
df = pd.concat([train_data, test_data], sort=True)

In [111]:
# Hianyzo adatok kitoltese
for att in df.columns:
    if att=="TARGET":
        pass
    #amik nem számok (string értékek stb...)
    elif df[att].dtype=="object": 
        #print(att)
        df[att]=df[att].fillna("nincs_adat")
    else:
        #test és train közös mediánja
        #ha van benne NaN, akkor azzal töltötte volna fel, de ez nem jó, ezért kell a nanmedian függvény
        #de nem a mediánra kéne itt kitölteni az fix!!!
        #df[att]=df[att].fillna(0) ez is lehetne
        df[att]=df[att].fillna(0)

In [112]:
# float64 to int64
for att in df.columns:
    if att =="TARGET":
        pass
    elif df[att].dtype == "float64":
        df[att] = df[att].astype('int64')

In [113]:
df.columns

Index(['AGE', 'AME_FLAG', 'APP_SUB_TYPE', 'BILL_AMOUNT_MONTH_1',
       'BILL_AMOUNT_MONTH_2', 'BILL_AMOUNT_MONTH_3', 'BILL_AMOUNT_MONTH_4',
       'BILL_AMOUNT_MONTH_5', 'BILL_AMOUNT_MONTH_6', 'BIRTH_CITY',
       'BIRTH_STATE', 'CARS_FLAG', 'CONF_CITIZEN_CARD_NUM', 'CONF_TAX_STATUS',
       'C_NAME_GIVEN', 'C_PRODUCT', 'DAY_OF_PAYMENT', 'EDU_LEVEL',
       'EMAIL_FLAG', 'HOME_PHONE_FLAG', 'ID', 'L_BALANCE', 'MATE_PROF_CODE',
       'MCARD_FLAG', 'MONTHS_IN_RES', 'M_IN_THE_JOB', 'M_STATUS', 'NAT',
       'NUMBERO_OF_BANK_ACCOUNTS', 'NUMBER_OF_DEPENDANTS',
       'NUMBER_OF_SPECIAL_BANK_ACCOUNTS', 'OCARDS_FLAG', 'OCC_TYPE',
       'O_INCOMES', 'PAY_AMOUNT_MONTH_1', 'PAY_AMOUNT_MONTH_2',
       'PAY_AMOUNT_MONTH_3', 'PAY_AMOUNT_MONTH_4', 'PAY_AMOUNT_MONTH_5',
       'PAY_AMOUNT_MONTH_6', 'PAY_MONTH_0', 'PAY_MONTH_2', 'PAY_MONTH_3',
       'PAY_MONTH_4', 'PAY_MONTH_5', 'PAY_MONTH_6', 'PROF_BOROUGH',
       'PROF_CITY', 'PROF_CODE', 'PROF_PHONE_AREA_CODE', 'PROF_PHONE_GIVEN',
       'PROF

In [114]:
bemeno_valtozok = ['AGE', 'AME_FLAG', 'APP_SUB_TYPE', 'BILL_AMOUNT_MONTH_1',
       'BILL_AMOUNT_MONTH_2', 'BILL_AMOUNT_MONTH_3', 'BILL_AMOUNT_MONTH_4',
       'BILL_AMOUNT_MONTH_5', 'BILL_AMOUNT_MONTH_6', 'CARS_FLAG', 'C_PRODUCT', 'DAY_OF_PAYMENT', 'EDU_LEVEL',
       'EMAIL_FLAG', 'HOME_PHONE_FLAG', 'L_BALANCE', 'MATE_PROF_CODE',
       'MCARD_FLAG', 'MONTHS_IN_RES', 'M_IN_THE_JOB', 'M_STATUS', 'NAT',
       'NUMBERO_OF_BANK_ACCOUNTS', 'NUMBER_OF_DEPENDANTS',
       'NUMBER_OF_SPECIAL_BANK_ACCOUNTS', 'OCARDS_FLAG', 'OCC_TYPE',
       'O_INCOMES', 'PAY_AMOUNT_MONTH_1', 'PAY_AMOUNT_MONTH_2',
       'PAY_AMOUNT_MONTH_3', 'PAY_AMOUNT_MONTH_4', 'PAY_AMOUNT_MONTH_5',
       'PAY_AMOUNT_MONTH_6', 'PAY_MONTH_0', 'PAY_MONTH_2', 'PAY_MONTH_3',
       'PAY_MONTH_4', 'PAY_MONTH_5', 'PAY_MONTH_6', 'PROF_BOROUGH', 'P_ADDRESS_TYPE', 'P_ASSETS_VALUE',
       'P_MONTHLY_INCOME', 'RES_BOROUGH', 'RES_TYPE', 'SEX', 'S_FLAG',
       'VISA_FLAG']

In [115]:
object_valtozok = []
uj_valtozok = []
for att in bemeno_valtozok:
    if df[att].dtype == 'object':
        df, val = limitalt_dummy_valtozo(df, att, 6)
        object_valtozok.append(att)
        uj_valtozok.extend(val)

In [116]:
bemeno_valtozok = bemeno_valtozok + uj_valtozok
bemeno_valtozok = [val for val in bemeno_valtozok if val not in object_valtozok]

In [117]:
d = []
for att in bemeno_valtozok:
    d.append(df[att].dtype)
d

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'

## Modellezés

In [118]:
train_df = df[df['train_or_test'] == 'train'].copy()
test_df = df[df['train_or_test'] == 'test'].copy()

In [119]:
train_df.columns

Index(['AGE', 'AME_FLAG', 'APP_SUB_TYPE', 'BILL_AMOUNT_MONTH_1',
       'BILL_AMOUNT_MONTH_2', 'BILL_AMOUNT_MONTH_3', 'BILL_AMOUNT_MONTH_4',
       'BILL_AMOUNT_MONTH_5', 'BILL_AMOUNT_MONTH_6', 'BIRTH_CITY',
       'BIRTH_STATE', 'CARS_FLAG', 'CONF_CITIZEN_CARD_NUM', 'CONF_TAX_STATUS',
       'C_NAME_GIVEN', 'C_PRODUCT', 'DAY_OF_PAYMENT', 'EDU_LEVEL',
       'EMAIL_FLAG', 'HOME_PHONE_FLAG', 'ID', 'L_BALANCE', 'MATE_PROF_CODE',
       'MCARD_FLAG', 'MONTHS_IN_RES', 'M_IN_THE_JOB', 'M_STATUS', 'NAT',
       'NUMBERO_OF_BANK_ACCOUNTS', 'NUMBER_OF_DEPENDANTS',
       'NUMBER_OF_SPECIAL_BANK_ACCOUNTS', 'OCARDS_FLAG', 'OCC_TYPE',
       'O_INCOMES', 'PAY_AMOUNT_MONTH_1', 'PAY_AMOUNT_MONTH_2',
       'PAY_AMOUNT_MONTH_3', 'PAY_AMOUNT_MONTH_4', 'PAY_AMOUNT_MONTH_5',
       'PAY_AMOUNT_MONTH_6', 'PAY_MONTH_0', 'PAY_MONTH_2', 'PAY_MONTH_3',
       'PAY_MONTH_4', 'PAY_MONTH_5', 'PAY_MONTH_6', 'PROF_BOROUGH',
       'PROF_CITY', 'PROF_CODE', 'PROF_PHONE_AREA_CODE', 'PROF_PHONE_GIVEN',
       'PROF

In [120]:
d = []
for att in bemeno_valtozok:
    d.append(train_df[att].dtype)
d

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'

In [121]:
# model1
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression(random_state=42, max_iter=10000)
auc = myxval(model1, train_df, bemeno_valtozok)

In [122]:
# model2
from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier(random_state=42, max_depth=5)
auc = myxval(model2, train_df, bemeno_valtozok)

In [123]:
# model3
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier(random_state=42, n_estimators=300, max_depth=15)
model3.fit(train_df[bemeno_valtozok], train_df['TARGET'])
prediction_rfc = model3.predict_proba(test_df[bemeno_valtozok])

In [124]:
prediction_rfc

array([[0.75095433, 0.24904567],
       [0.70289636, 0.29710364],
       [0.48679738, 0.51320262],
       ...,
       [0.59969358, 0.40030642],
       [0.49595869, 0.50404131],
       [0.59073919, 0.40926081]])

In [125]:
# model4
from sklearn.ensemble import GradientBoostingClassifier
model4 = GradientBoostingClassifier(random_state=42, n_estimators=150, max_depth=10)
model4.fit(train_df[bemeno_valtozok], train_df['TARGET'])
prediction_gbc = model4.predict_proba(test_df[bemeno_valtozok])

In [126]:
prediction_gbc

array([[0.75138268, 0.24861732],
       [0.70143491, 0.29856509],
       [0.14375116, 0.85624884],
       ...,
       [0.72168684, 0.27831316],
       [0.18834643, 0.81165357],
       [0.59680572, 0.40319428]])

In [127]:
rfc_p1 = prediction_rfc[:, 1]
gbc_p1 = prediction_gbc[:, 1]

In [128]:
a = np.array([rfc_p1, gbc_p1])

In [129]:
a

array([[0.24904567, 0.29710364, 0.51320262, ..., 0.40030642, 0.50404131,
        0.40926081],
       [0.24861732, 0.29856509, 0.85624884, ..., 0.27831316, 0.81165357,
        0.40319428]])

In [130]:
b = np.mean(a, axis=0)

In [131]:
test_df['mean_p1'] = b

In [132]:
submission_df = test_df[['ID', 'mean_p1']]
submission_df.columns = ['Id', 'Predicted']
submission_df.to_csv('proba_mean.csv', index=False)

In [133]:
myxval_multi([model3, model4], train_df, bemeno_valtozok)

0.7486386563717712